In [4]:
import math
import plotly.plotly as py
from plotly.tools import FigureFactory as FF
import plotly.graph_objs as go

import numpy as np
import pandas as pd
import scipy


Case air ordi

In [5]:
def T_air_ordi(Tcd, Tcv, Tcase):
    """  Float^3 -> Float
    rend la température du volume d'air possedant un ordinateur
    """
    C = 1.005
    VI = 1 
    Vi = VI - 0.3
    M2 = (1.292*(273.15/(Tcv+273.15)))*0.3
    M1 = 1.292*(273.15/(Tcv + 273.15))*0.15
    Ma = (1.202*(273.15/Tcase))*Vi
    S = 0.025
    k =0.025
    dx = 0.001
    dt = 2
    
    Tfcv = (math.exp((-dt)/(C*M1))/(C*M1))+Tcv
    Tfcd = (dx/(k*S))*(math.exp(0.15/S))+Tcd
    
    TF = ((Ma*C*Tcase)+(M1*C*Tfcv)+(M2*C*Tfcd))/(C*(Ma+M1+M2))
    return TF

In [6]:
TCd = 40
TCv = 30
TAmb = 20

T_air_ordi(TCd, TCv, TAmb)

39.488711086105425

transfert case/case

In [7]:
def T_new_case(Tcase1, Tcase2):
    """ Float*Float -> Float
    rend température air d'une case après tranfert de température
    """
    
    C = 1.005
    VI = 1
    M1 = (1.292*(273.15/Tcase1))*VI
    Ma = (1.292*(273.15/Tcase2))*VI
    dt = 2
    
    Tf = (math.exp((-dt)/(C*M1))/(C*M1))+Tcase1
    
    TF = (Ma*C*Tcase2+M1*C*Tf)/(C*(Ma+M1))
    
    return TF

In [8]:
T1 = T_air_ordi(TCd, TCv, TAmb)
T2 = 20

T_new_case(T1, T2)

26.582029681103105

Modèlisation matrice sans ventilation

In [40]:
def modèle(T_air_ordi, Tamb):
    """Float*Float -> matrix
    rend le modèle de la salle
    """
   
    MS = { 10:Tamb, 11:Tamb, 12:Tamb, 13:Tamb, 14:Tamb, 15:Tamb, 16:Tamb, 17:Tamb, 18:Tamb, 19:Tamb,
           20:Tamb, 21:Tamb, 22:Tamb, 23:Tamb, 24:Tamb, 25:Tamb, 26:Tamb, 27:Tamb, 28:Tamb, 29:Tamb,
           30:Tamb, 31:Tamb, 32:Tamb, 33:Tamb, 34:Tamb, 35:Tamb, 36:Tamb, 37:Tamb, 38:Tamb, 39:Tamb,
           40:Tamb, 41:Tamb, 42:Tamb, 43:Tamb, 44:Tamb, 45:Tamb, 46:Tamb, 47:Tamb, 48:Tamb, 49:Tamb,
           50:Tamb, 51:Tamb, 52:Tamb, 53:Tamb, 54:Tamb, 55:Tamb, 56:Tamb, 57:Tamb, 58:Tamb, 59:Tamb}
    
    A = [21, 22, 23, 2, 25, 26, 27, 28, 31, 32, 33, 34, 34, 36, 37, 38, 41, 42, 43, 44, 45, 46, 47, 48]
    B = [20, 30, 40]
    C = [11, 12, 13, 14, 15, 16, 17, 18]
    D = [29, 39, 49]
    E = [51, 52, 53, 54, 55, 56, 57, 58]
    
    MS[34] = 0
    for j in range (10, 60) :
        if MS[j] == 0 :
            MS[j] = T_air_ordi
            i = j
            while i < 60  and i >9 :
                
                if i in A :
                    if MS[i+10] == Tamb :
                        MS[i+10] = T_new_case(MS[i], MS[i+10])
                    if MS[i-10] == Tamb :
                        MS[i-10] = T_new_case(MS[i], MS[i-10])
                    if MS[i+1] == Tamb :
                        MS[i+1] = T_new_case(MS[i], MS[i+1])
                    if MS[i-1] == Tamb :
                        MS[i-1] = T_new_case(MS[i], MS[i-1])
                elif i in B :
                    if MS[i+10] == Tamb :
                        MS[i+10] = T_new_case(MS[i], MS[i+10])
                    if MS[i+1] == Tamb :
                        MS[i+1] = T_new_case(MS[i], MS[i+1])
                    if MS[i-10] == Tamb :
                        MS[i-10] = T_new_case(MS[i], MS[i-10])
                elif i in C :
                    if MS[i+10] == Tamb :
                        MS[i+10] = T_new_case(MS[i], MS[i+10])
                    if MS[i+1] == Tamb :
                        MS[i+1] = T_new_case(MS[i], MS[i+1])
                    if MS[i-1] == Tamb :
                        MS[i-1] = T_new_case(MS[i], MS[i-1])
                elif i in D :
                    if MS[i+10] == Tamb :
                        MS[i+10] = T_new_case(MS[i], MS[i+10])
                    if MS[i-1] == Tamb :
                        MS[i-1] = T_new_case(MS[i], MS[i-1])
                    if MS[i-10] == Tamb :
                        MS[i-10] = T_new_case(MS[i], MS[i-10])
                elif i in E :
                    if MS[i-10] == Tamb :
                        MS[i-10] = T_new_case(MS[i], MS[i-10])
                    if MS[i+1] == Tamb :
                        MS[i+1] = T_new_case(MS[i], MS[i+1])
                    if MS[i-1] == Tamb :
                        MS[i-1] = T_new_case(MS[i], MS[i-1])
                elif i == 10 :
                    if MS[i+10] == Tamb :
                        MS[i+10] = T_new_case(MS[i], MS[i+10])
                    if MS[i+1] == Tamb :
                        MS[i+1] = T_new_case(MS[i], MS[i+1])
                elif i == 19 :
                    if MS[i+10] == Tamb :
                        MS[i+10] = T_new_case(MS[i], MS[i10])
                    if MS[i-1] == Tamb :
                        MS[i-1] = T_new_case(MS[i], MS[i-1])
                elif i == 59 :
                    if MS[i-1] == Tamb :
                        MS[i-1] = T_new_case(MS[i], MS[i-1])
                    if MS[i-10] == Tamb :
                        MS[i-10] = T_new_case(MS[i], MS[i-10])
                elif i == 50 :
                    if MS[i-10] == Tamb :
                        MS[i-10] = T_new_case(MS[i], MS[i-10])
                    if MS[i+1] == Tamb :
                        MS[i+1] = T_new_case(MS[i], MS[i+1])
                i = i-1
                
                    
    return np.matrix(
                [[MS[10], MS[11], MS[12], MS[13], MS[14], MS[15], MS[16], MS[17], MS[18], MS[19]],
                 [MS[20], MS[21], MS[22], MS[23], MS[24], MS[25], MS[26], MS[27], MS[28], MS[29]],
                 [MS[30], MS[31], MS[32], MS[33], MS[34], MS[35], MS[36], MS[37], MS[38], MS[39]],
                 [MS[40], MS[41], MS[42], MS[43], MS[44], MS[45], MS[46], MS[47], MS[48], MS[49]],
                 [MS[50], MS[51], MS[52], MS[53], MS[54], MS[55], MS[56], MS[57], MS[58], MS[59]]]
                )

In [41]:
Tamb = 20

modèle(T_air_ordi(TCd, TCv, TAmb), Tamb)

matrix([[20.20430009, 20.36110803, 20.68264657, 21.35826761, 20.04957977,
         20.04879584, 20.04722574, 20.04408144, 20.03778609, 20.02518783],
        [20.36110803, 20.68264657, 21.35826761, 22.85369468, 26.58202968,
         20.04722574, 20.04408144, 20.03778609, 20.02518783, 20.03778609],
        [20.68264657, 21.35826761, 22.85369468, 26.58202968, 39.48871109,
         26.58202968, 20.04722574, 20.04408144, 20.03778609, 20.02518783],
        [20.36110803, 20.68264657, 21.35826761, 22.85369468, 26.58202968,
         20.        , 20.        , 20.        , 20.        , 20.        ],
        [20.        , 20.        , 20.        , 20.        , 20.        ,
         20.        , 20.        , 20.        , 20.        , 20.        ]])

In [0]:
        [20.        , 20.        , 20.        , 20.        , 20.        , 20.        , 20.        , 20.        , 20.        , 20.        ],
        [20.        , 20.        , 20.        , 20.        , 26.58202968, 20.        , 20.02518783, 20.03778609, 20.04408144, 20.04722574],
        [20.02518783, 20.03778609, 20.04408144, 26.58202968, 39.48871109, 26.58202968, 20.03778609, 20.02518783, 20.03778609, 20.04408144],
        [20.03778609, 20.02518783, 20.03778609, 20.04408144, 26.58202968, 22.85369468, 20.02518783, 20.03778609, 20.04408144, 20.04722574],
        [20.02518783, 20.03778609, 20.04408144, 20.04722574, 22.85369468, 21.35826761, 20.03778609, 20.04408144, 20.04722574, 20.04879584]